# Generate carve.me model which builds model.xml files using FASTA RNA files
- Requires completion of "check__", "confirm__", and "create_community__" files

In [ ]:
# Loading libraries
import pandas as pd
import re
import cobra
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium

# This dataframe represent the compounds found on the minimal media of the model.
filtered_df = pd.read_csv("concerto/concerto/memotedata/CarveMeMinimalMediaFile.csv")
#print(filtered_df)
# This dataframe represents the BiGG reactions found on the biolog file.
biolog_names = pd.read_csv("concerto/concerto/memotedata/PM02OnlyNamesAndBiGG.csv")
#print(biolog_names)

'''This code will help you create a tsv file to use on the carveme software'''
- carve.me builds model.mxl files, includes gapfill function which addes exchanged rxns (carbon sources) to model
- ASK: what are we doing with the metabolite parts from BiGG reactions after we extract them?
- Why i = 12, does the model run max out at 12?
- TODO: figure out exaclty what previous dataframes we need for this that are going in new_row
  

In [ ]:
new_rows = [] # We create an empty list

for j in range(len(biolog_names["BiGG"])): #We iterate through each reaction of each carbon source metabolite from the biolog file.
    #Here we just extract the metabolite part from the BiGG reaction ID. In example: EX_csc_e --> csc
    bigg_name = re.sub("EX_", "", biolog_names["BiGG"][j])
    bigg_name = re.sub("_e", "", bigg_name)
    # We iterate through each of the compounds found on the biolog file.
    for i in range(len(filtered_df["compound"])):
        #Here we add create a new row with the information found on the previous dataframes.
        new_row = {
            "medium": "MM_[" + bigg_name + "]", #Here we create the name of the medium. Ex: MM_[R_actn]
            "description": "Minimal medium (" + biolog_names["REAGENT"][j] + ")", #Here we specify a description of the minimal media.
            "compound": filtered_df["compound"].iloc[i], #Here we add each of the compounds found on the minimal media of the model.
            "name": filtered_df["name"].iloc[i] #Here we add each name of the compounds from the minimal media.
        }
        new_rows.append(new_row) # We append these new rows to the empty list.
        
        if i == 12: #If i is equal to 12 then we add a row containing the carbon source from the biolog file.
            biolog_row = {
                "medium": "MM_[" + bigg_name + "]",
                "description": "Minimal medium (" + biolog_names["REAGENT"][j] + ")",
                "compound": bigg_name,
                "name": biolog_names["REAGENT"][j]
            }
            new_rows.append(biolog_row) # We append it to the list.

new_rows_df = pd.DataFrame(new_rows) #We transform the list into a dataframe.
filtered_df = pd.concat([filtered_df, new_rows_df], ignore_index=True) #We concatenate the new dataframe to the filtered_df file.

filtered_df.to_csv("CarveMeDBV3.tsv", sep="\t", index=False) # we create a tsv file with the final dataframe.
print(filtered_df)

   medium    description compound               name
0      MM  Minimal Media      ca2            Calcium
1      MM  Minimal Media       cl           Chloride
2      MM  Minimal Media  cobalt2               Co2+
3      MM  Minimal Media      cu2             Copper
4      MM  Minimal Media      fe2  Fe2+ mitochondria
5      MM  Minimal Media        k          Potassium
6      MM  Minimal Media      mg2          Magnesium
7      MM  Minimal Media      mn2          Manganese
8      MM  Minimal Media      no3            Nitrate
9      MM  Minimal Media       o2              O2 O2
10     MM  Minimal Media      so4            Sulfate
11     MM  Minimal Media     tsul        Thiosulfate
12     MM  Minimal Media      zn2               Zinc
                     REAGENT            BiGG
0      Chondroitin Sulfate C       EX_cs_c_e
1                    Dextrin    EX_dextrin_e
2                   Glycogen   EX_glycogen_e
3                     Mannan     EX_mannan_e
4                     Pectin     